In [ ]:
!unzip "/content/sample_data/BAU-Insectv2.zip"

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Data Preparation
data_dir = "/content/extracted_folder/pest"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

# Optimized Data Augmentation
data_gen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_data = data_gen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='training')
val_data = data_gen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='validation')

# Load VGG16 & Unfreeze Top Layers
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg_base.layers[:-4]:  # Unfreezing last 4 layers for fine-tuning
    layer.trainable = True

# Improved Model Architecture
model = Sequential([
    vgg_base,
    Flatten(),
    Dense(512),
    BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    Dropout(0.5),
    Dense(256),
    BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    Dropout(0.4),
    Dense(128),
    BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    Dropout(0.3),
    Dense(train_data.num_classes, activation='softmax')
])

# Compile Model with Optimized Learning Rate
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(train_data, validation_data=val_data, epochs=30, batch_size=32)

# Evaluation on Test Data
class_labels = list(train_data.class_indices.keys())
y_true, y_pred = [], []

for class_name in os.listdir(test_dir):
    class_dir = os.path.join(test_dir, class_name)
    if os.path.isdir(class_dir):
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            try:
                img = image.load_img(image_path, target_size=(224, 224))
                x = image.img_to_array(img) / 255.0
                x = np.expand_dims(x, axis=0)
                classes = model.predict(x)
                predicted_class = np.argmax(classes[0])
                y_true.append(class_labels.index(class_name))
                y_pred.append(predicted_class)
            except Exception as e:
                print(f"Error processing image {image_path}: {e}")

# Classification Report
print(classification_report(y_true, y_pred, target_names=class_labels))


Archive:  /content/sample_data/BAU-Insectv2.zip
   creating: extracted_folder/
   creating: extracted_folder/pest/
   creating: extracted_folder/pest/test/
   creating: extracted_folder/pest/test/aphids/
  inflating: extracted_folder/pest/test/aphids/jpg_0 - Copy.jpg  
  inflating: extracted_folder/pest/test/aphids/jpg_0.jpg  
  inflating: extracted_folder/pest/test/aphids/jpg_13 - Copy.jpg  
  inflating: extracted_folder/pest/test/aphids/jpg_13.jpg  
  inflating: extracted_folder/pest/test/aphids/jpg_15 - Copy.jpg  
  inflating: extracted_folder/pest/test/aphids/jpg_15.jpg  
  inflating: extracted_folder/pest/test/aphids/jpg_19.jpg  
  inflating: extracted_folder/pest/test/aphids/jpg_2 - Copy.jpg  
  inflating: extracted_folder/pest/test/aphids/jpg_2.jpg  
  inflating: extracted_folder/pest/test/aphids/jpg_23.jpg  
  inflating: extracted_folder/pest/test/aphids/jpg_24.jpg  
  inflating: extracted_folder/pest/test/aphids/jpg_27.jpg  
  inflating: extracted_folder/pest/test/aphids/jpg_2

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.1224 - loss: 2.5714 - val_accuracy: 0.1351 - val_loss: 5.2456
Epoch 2/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 53s 717ms/step - accuracy: 0.1619 - loss: 2.4140 - val_accuracy: 0.0833 - val_loss: 5.3686
Epoch 3/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 40s 711ms/step - accuracy: 0.1804 - loss: 2.3080 - val_accuracy: 0.1261 - val_loss: 4.7573
Epoch 4/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 40s 705ms/step - accuracy: 0.2119 - loss: 2.2205 - val_accuracy: 0.1532 - val_loss: 4.9092
Epoch 5/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 40s 709ms/step - accuracy: 0.2445 - loss: 2.1383 - val_accuracy: 0.1869 - val_loss: 2.9436
Epoch 6/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 39s 698ms/step - accuracy: 0.2825 - loss: 2.0198 - val_accuracy: 0.1081 - val_loss: 5.3824
Epoch 7/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 41s 705ms/step - accuracy: 0.2795 - loss: 1.9827 - val_accuracy: 0.2117 - val_loss: 2.8869
Epoch 8/30
56/56 ━━━━━━━━━━━━━━━━━━━━ 39s 700ms/step - accuracy: 0.3448 - loss: 1.8366 - val_accura

In [ ]:
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.8671875


In [ ]:
# prompt: SAVE MODEL

model.save('my_model.h5')
!zip -r my_model.zip my_model.h5
from google.colab import files
files.download('my_model.zip')


  adding: my_model.h5 (deflated 10%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>